### 학습목표
     * 시각화를 해보고, 모델을 개선시킨다.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import warnings
warnings.filterwarnings("ignore")

In [4]:
train = pd.read_csv('../../../data/Parking_Demand/train.csv')
test = pd.read_csv('../../../data/Parking_Demand/test.csv')
age = pd.read_csv('../../../data/Parking_Demand/age_gender_info.csv')
sub = pd.read_csv('../../../data/Parking_Demand/sample_submission.csv')

train.shape, test.shape, age.shape

((2952, 15), (1022, 14), (16, 23))

In [5]:
train.columns, test.columns

(Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
        '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
        '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
       dtype='object'),
 Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
        '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
        '도보 10분거리 내 버스정류장 수', '단지내주차면수'],
       dtype='object'))

### 컬럼명이 길다보니 다음과 같이 컬럼명의 길이를 간소화 해주자

In [6]:
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수', '등록차량수']
test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수']

### 결측치 처리(1)

In [7]:
train.isna().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         569
임대료           569
10분내지하철수      211
10분내버스정류장수      4
단지내주차면수         0
등록차량수           0
dtype: int64

In [8]:
test.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         180
임대료           180
10분내지하철수       42
10분내버스정류장수      0
단지내주차면수         0
dtype: int64

In [9]:
test.loc[test['자격유형'].isnull()] 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0


In [10]:
grouped = test.groupby(['단지코드', '임대건물구분', '지역','공급유형'])
group1 = grouped.get_group( ('C2411', '아파트', '경상남도', '국민임대')  )
group1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000,100720,0.0,2.0,840.0
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000,100720,0.0,2.0,840.0
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000,100720,0.0,2.0,840.0
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000,171480,0.0,2.0,840.0


In [11]:
test.loc[196,'자격유형'] = 'A'

In [12]:
group2 = grouped.get_group( ('C2253', '아파트', '강원도', '영구임대')  )
group2

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000,83020,0.0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000,83020,0.0,2.0,173.0


In [13]:
test.loc[258,'자격유형'] = 'C'

In [14]:
print(train.자격유형.unique())
print(test.자격유형.unique())

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
['H' 'A' 'E' 'C' 'D' 'G' 'I' 'J' 'K' 'L' 'M' 'N']


In [15]:
mapping = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 
           'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 
           'K':11, 'L':12, 'M':13, 'N':14, 'O':15}
train['자격유형'] = train['자격유형'].map(mapping).astype(int)
test['자격유형'] = test['자격유형'].map(mapping).astype(int)

train.head(3), test.head(3)

(    단지코드  총세대수 임대건물구분    지역  공급유형   전용면적  전용면적별세대수   공가수  자격유형     임대보증금  \
 0  C2483   900    아파트  경상북도  국민임대  39.72       134  38.0     1  15667000   
 1  C2483   900    아파트  경상북도  국민임대  39.72        15  38.0     1  15667000   
 2  C2483   900    아파트  경상북도  국민임대  51.93       385  38.0     1  27304000   
 
       임대료  10분내지하철수  10분내버스정류장수  단지내주차면수   등록차량수  
 0  103680       0.0         3.0   1425.0  1015.0  
 1  103680       0.0         3.0   1425.0  1015.0  
 2  184330       0.0         3.0   1425.0  1015.0  ,
     단지코드  총세대수 임대건물구분   지역  공급유형   전용면적  전용면적별세대수   공가수  자격유형     임대보증금  \
 0  C1072   754    아파트  경기도  국민임대  39.79       116  14.0     8  22830000   
 1  C1072   754    아파트  경기도  국민임대  46.81        30  14.0     1  36048000   
 2  C1072   754    아파트  경기도  국민임대  46.90       112  14.0     8  36048000   
 
       임대료  10분내지하철수  10분내버스정류장수  단지내주차면수  
 0  189840       0.0         2.0    683.0  
 1  249930       0.0         2.0    683.0  
 2  249930       0.0         2.0    683.0  

In [16]:
test.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,8,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,1,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,8,36048000,249930,0.0,2.0,683.0


In [17]:
test['공급유형'].unique()

array(['국민임대', '영구임대', '임대상가', '공공임대(50년)', '공공임대(10년)', '공공임대(분납)',
       '행복주택'], dtype=object)

In [18]:
test['공급유형'].unique()

array(['국민임대', '영구임대', '임대상가', '공공임대(50년)', '공공임대(10년)', '공공임대(분납)',
       '행복주택'], dtype=object)

### 라벨인코딩

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
le = LabelEncoder()
le = le.fit(train['공급유형'])
train['공급유형'] = le.transform(train['공급유형'])
test['공급유형'] = le.transform(test['공급유형'])

In [21]:
train.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,5,39.72,134,38.0,1,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,5,39.72,15,38.0,1,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,5,51.93,385,38.0,1,27304000,184330,0.0,3.0,1425.0,1015.0


### test['지역']의 '서울특별시'를 제외한 후 지역 라벨인코딩

In [43]:
test_d = test[test['지역']=='서울특별시'].index
test = test.drop(test_d)
test.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,단지별차량수평균
0,C1072,754,아파트,1,5,39.79,116,14.0,8,22830000,189840,0.0,2.0,683.0,641.57,651.89125
1,C1072,754,아파트,1,5,46.81,30,14.0,1,36048000,249930,0.0,2.0,683.0,659.07,651.89125
2,C1072,754,아파트,1,5,46.90,112,14.0,8,36048000,249930,0.0,2.0,683.0,645.50,651.89125


In [44]:
le2 = LabelEncoder()
le2 = le.fit(train['지역'])
train['지역'] = le.transform(train['지역'])
test['지역'] = le.transform(test['지역'])

In [45]:
sel = ['총세대수', '전용면적', '전용면적별세대수', '공가수',  '단지내주차면수', '자격유형','지역', '공급유형']
X_train = train[sel]
X_test = test[sel]
y_train = train['등록차량수']

In [46]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [47]:
test['등록차량수'] = pred
test['단지별차량수평균'] = test.groupby("단지코드")['등록차량수'].transform(np.mean)
test_new = test.drop_duplicates(['단지코드'], keep='first').reset_index()
test_new

,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,단지별차량수평균
0,0,C1072,754,아파트,1,5,39.79,116,14.0,8,22830000,189840,0.0,2.0,683.0,641.57,651.891250
1,8,C1128,1354,아파트,1,5,39.79,368,9.0,8,22830000,189840,0.0,3.0,1216.0,1162.54,1167.724444
2,17,C1456,619,아파트,7,5,33.40,82,18.0,1,19706000,156200,0.0,16.0,547.0,583.11,602.034444
3,26,C1840,593,아파트,12,5,39.57,253,7.0,1,14418000,108130,0.0,3.0,543.0,458.76,451.655000
4,30,C1332,1297,아파트,1,5,39.99,282,11.0,8,28598000,203050,0.0,2.0,1112.0,999.55,1004.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,996,C2456,349,아파트,13,5,26.44,24,17.0,8,6992000,117000,0.0,4.0,270.0,240.64,250.102500
146,1000,C1266,596,아파트,15,5,26.94,164,35.0,8,8084000,149910,0.0,1.0,593.0,511.23,516.328000
147,1005,C2152,120,아파트,0,6,24.83,66,9.0,3,-,-,0.0,1.0,40.0,42.83,43.205000
148,1007,C1267,675,아파트,2,5,24.87,28,38.0,8,6882000,104370,0.0,1.0,467.0,468.77,435.455455


In [48]:
sub_df = test_new[ ['단지코드', '단지별차량수평균']]
sub_df.columns = ['code', 'num']
sub_df.to_csv('third_0712.csv', index=False)
sub_df.head()

,code,num
0,C1072,651.891250
1,C1128,1167.724444
2,C1456,602.034444
3,C1840,451.655000
4,C1332,1004.020000
